# Playground

This is a space for experimentation.

In [48]:
import random
import itertools
import numpy as np
from player import Player
from minimax import MiniMax
from datetime import datetime
from utility import int2board
from utility import board2int
from tic_tac_toe import WorldTTT
from strategies import StrategyTabQLearning

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Tic Tac Toe

### Check World Implementation

In [2]:
world_ttt = WorldTTT(
    name="ttt", board_size=(3, 3), 
    player1sym='X', player2sym='O'
)

In [14]:
board_test_ttt = np.array([
    [1, 0, 1],
    [-1, 0, 0],
    [1, 1, 0]
])
is_player1 = True
is_my_turn_next = True
action = ((1, 0), 1)
board_test_ttt

array([[ 1,  0,  1],
       [-1,  0,  0],
       [ 1,  1,  0]])

In [52]:
# # Test function get_actions(...)
# world_ttt.get_actions(is_player1)
# # [OK]

In [83]:
# # Test function is_valid(...)
# world_ttt.is_valid(board_test_ttt, is_player1)
# # [OK]

In [59]:
# # Test function is_legal(...)
# world_ttt.is_legal(board_test_ttt, ((0, 0), 1))
# # [OK]

In [77]:
# # Test get_next_state(...)
# next_state = world_ttt.get_next_state(board_test_ttt, action)
# print("Next State =", next_state)
# if not next_state == -1:
#     print("=", int2board(next_state, world_ttt.board.shape))
# # [OK]

In [88]:
# # Test state_eval(...)
# world_ttt.state_eval(board_test_ttt, is_my_turn_next)
# # [OK]

In [41]:
# # Test get_next_states(...)
# for s_num in world_ttt.get_next_states(
#     board_test_ttt, 
#     is_player1
# ): print(int2board(s_num[0], world_ttt.board.shape), '\n')
# # [OK]

In [113]:
# # Test get_reward(...)
# world_ttt.get_reward(board_test_ttt, action)
# # [OK]

In [96]:
# # Test is_winner(...)
# world_ttt.is_winner(board_test_ttt)
# # [OK]

### Check Q Learning Implementation

In [49]:
strategy_tabq = StrategyTabQLearning(
    get_reward=world_ttt.get_reward,
    is_game_over=world_ttt.is_game_over,
    get_next_states=world_ttt.get_next_states,
    get_next_state=world_ttt.get_next_state,
    get_actions=world_ttt.get_actions,
    get_start_states=world_ttt.get_start_states,
    board_shape=world_ttt.board.shape
) # Player 1 tabular Q learning strategy TTT.

In [54]:
strategy_tabq.get_move(
    board=board_test_ttt,
    is_player1=is_player1
)

{'f_out': (1, 0), 'seconds': 0.0}